In [2]:
# Import Required Libraries
import requests
import plotly.express as px
import pandas as pd
import numpy as np
from scipy.optimize import curve_fit
from sklearn.metrics import mean_squared_error

In [3]:
# Authenticate and Retrieve Portfolio

import robin_stocks.robinhood as r

# Define a function to authenticate with Robinhood API
def authenticate_robinhood(username, password):
    login = r.login(username, password)
    return login

# Define a function to retrieve the current portfolio tickers
def get_portfolio_tickers():
    positions_data = r.account.build_holdings()
    tickers = list(positions_data.keys())
    return tickers

['META', 'AAPL', 'SPOT', 'NVDA', 'SHOP', 'GOOGL', 'BRK.B', 'RBLX', 'GS', 'Z', 'LLY', 'AVAV', 'ARM', 'SMR', 'VRSK', 'JPM', 'TGS', 'MU', 'CEG']


In [190]:
from datetime import timezone


def linear_func(x, m, c):
    return m * x + c

def exponential_func(x, a, b):
    return a * np.exp(b * x)


#Define a function to calculate metrics for each ticker for a given time range
def calculate_regression_metrics_for_range(historical_data, start_date):
    results = []
    for ticker, data in historical_data.items():
        
        # Convert start_date to the same timezone as data.index
        start_date = pd.to_datetime(start_date).tz_convert(data.index.tz)
        
        # Filter data for the given time range
        data_range = data[data.index >= start_date]
        
        if data_range.empty:
            continue
        
        # Convert date to ordinal for fitting
        data_range['date'] = data_range.index.to_series().apply(lambda date: date.toordinal())
        x_data = data_range['date']
        y_data = data_range['close_price'].astype(float)

        # Fit the linear curve
        popt_linear = np.polyfit(x_data, y_data, 1)
        
        # Fit the exponential curve
        popt_exp, _ = curve_fit(exponential_func, x_data, y_data, maxfev=10000)
        
        # Calculate metrics for linear fit
        y_pred_linear = linear_func(x_data, *popt_linear)
        slope = popt_linear[0]
        variance = np.var(y_pred_linear)
        std_dev = np.std(y_pred_linear)
        max_drawdown = np.min(y_pred_linear) - np.max(y_pred_linear)
        
        # Calculate metrics for exponential fit
        exp_param = popt_exp[1]  # Exponential growth rate
        
        # Append results
        results.append({
            'ticker': ticker,
            'slope': slope,
            'exp_param': exp_param,
            'variance': variance,
            'standard_deviation': std_dev,
            'maximum_drawdown': max_drawdown
        })
    

    
    return pd.DataFrame(results)

# Define a function to calculate metrics for multiple time ranges
def calculate_metrics_for_multiple_ranges(historical_data):
    time_ranges = {
        '5_years': datetime.now(tz=timezone.utc) - timedelta(days=5*365),
        '3_years': datetime.now(tz=timezone.utc) - timedelta(days=3*365),
        '1_year': datetime.now(tz=timezone.utc) - timedelta(days=365),
        '6_months': datetime.now(tz=timezone.utc) - timedelta(days=6*30),
        '3_months': datetime.now(tz=timezone.utc) - timedelta(days=3*30),
        '1_week': datetime.now(tz=timezone.utc) - timedelta(days=7),
        '3_days': datetime.now(tz=timezone.utc) - timedelta(days=3)
    }
    aggregated_results = pd.DataFrame()
    
    for range_name, start_date in time_ranges.items():
        metrics_df = calculate_regression_metrics_for_range(historical_data, start_date)
        metrics_df['time_range'] = range_name
        aggregated_results = pd.concat([aggregated_results, metrics_df], ignore_index=True)
    
    return aggregated_results



In [191]:
import os
import time
import pickle
from datetime import datetime, timedelta

# Fetch Historical Data

# Define a function to fetch historical data for a given ticker
def fetch_historical_data(ticker, interval='day', span='5year'):
    time.sleep(1)
    historical_data = r.stocks.get_stock_historicals(ticker, interval=interval, span=span)
    historical_data = pd.DataFrame(historical_data)
    historical_data.index = pd.to_datetime(historical_data['begins_at']).dt.tz_convert('UTC')
    return historical_data

# Define a function to load data from cache if it exists and is younger than 1 day
def load_from_cache(ticker):
    print("Loading from cache")
    cache_file = f"{ticker}_historical_data.pkl"
    if os.path.exists(cache_file):
        file_time = datetime.fromtimestamp(os.path.getmtime(cache_file))
        if datetime.now() - file_time < timedelta(days=1):
            with open(cache_file, 'rb') as f:
                return pickle.load(f)
    return None

# Define a function to save data to cache
def save_to_cache(ticker, data):
    print("Saving to cache")
    cache_file = f"{ticker}_historical_data.pkl"
    with open(cache_file, 'wb') as f:
        pickle.dump(data, f)


In [192]:
def calulate_time_slope_delta_plot(grouped_data, group_title):

    # Calculate metrics for multiple time ranges
    aggregated_results = calculate_metrics_for_multiple_ranges(grouped_data)

    pivoted_results = aggregated_results.pivot(index='ticker', columns='time_range', values=['slope', 'exp_param', 'variance', 'standard_deviation', 'maximum_drawdown'])
    pivoted_results

    aggregated_results.sort_values('slope', ascending=False)

    # Sort the time ranges
    time_ranges_order = ['3_days', '1_week', '3_months', '6_months', '1_year', '3_years', '5_years']
    aggregated_results['time_range'] = pd.Categorical(aggregated_results['time_range'], categories=time_ranges_order, ordered=True)
    sorted_results = aggregated_results.sort_values('time_range', ascending=False)

    # Create the plot
    fig = px.line(sorted_results, x='time_range', y='slope', color='ticker', markers=True)

    fig.update_layout(
        title=f"Slope of Different Tickers Over Various Time Ranges {group_title}",
        xaxis_title="Time Range",
        yaxis_title="Slope"
    )

    return fig, sorted_results


In [ ]:
# Example usage
username = input("Enter your Robinhood username: ")
password = input("Enter your Robinhood password: ")
authenticate_robinhood(username, password)

In [193]:
def run_stats_for_group(group_name):

    if group_name == 'portfolio':
        tickers = get_portfolio_tickers()
        print(tickers)

        historical_data = {}
        for ticker in tickers:
            data = load_from_cache(ticker)
            if data is None:
                data = fetch_historical_data(ticker)
                save_to_cache(ticker, data)
            historical_data[ticker] = data

    else:
        tickers = stock_lists[group_name]
        print(tickers)

        historical_data = {}
        for ticker in tickers:
            data = load_from_cache(ticker)
            if data is None:
                data = fetch_historical_data(ticker)
                save_to_cache(ticker, data)
            historical_data[ticker] = data
    
    return historical_data

In [178]:
slope_df['time_range'].unique()

['5_years', '3_years', '1_year', '6_months', '3_months', '1_week']
Categories (7, object): ['1_day' < '1_week' < '3_months' < '6_months' < '1_year' < '3_years' < '5_years']

In [194]:
historical_data = {}

historical_data['portfolio'] = run_stats_for_group("portfolio")
fig, slope_df = calulate_time_slope_delta_plot(historical_data['portfolio'], group_title="Portfolio")

['META', 'AAPL', 'SPOT', 'NVDA', 'SHOP', 'GOOGL', 'BRK.B', 'RBLX', 'GS', 'Z', 'LLY', 'AVAV', 'ARM', 'SMR', 'VRSK', 'JPM', 'TGS', 'MU', 'CEG', 'KVYO']
Loading from cache
Loading from cache
Loading from cache
Loading from cache
Loading from cache
Loading from cache
Loading from cache
Loading from cache
Loading from cache
Loading from cache
Loading from cache
Loading from cache
Loading from cache
Loading from cache
Loading from cache
Loading from cache
Loading from cache
Loading from cache
Loading from cache
Loading from cache


/tmp/ipykernel_1221821/142773724.py:26: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [195]:
fig.show()

In [76]:
# Define a function to get all stock lists from Robinhood
def get_all_stock_lists():
    watchlists = r.account.get_all_watchlists()['results']
    stock_lists = {}
    for watchlist in watchlists:
        try:
            list_name = watchlist['display_name']
            print(list_name)
            results = r.account.get_watchlist_by_name(list_name)['results']
            symbols = [item['symbol'] for item in results]
            stock_lists[list_name] = symbols
        except Exception as e:
            print(f"Error fetching watchlist {list_name}: {e}")
    return stock_lists

# Example usage
stock_lists = get_all_stock_lists()




Post Stonks
Dividend
Biotech
Energy
Options Watchlist
400 Client Error: Bad Request for url: https://api.robinhood.com/midlands/lists/items/?list_id=633d6309-6671-4fb2-95bb-35357476d1d2
Error fetching watchlist Options Watchlist: 'NoneType' object is not subscriptable
Bonds
Banks, Finance and Homes
High debt load
Massachusetts
Daily Movers
QT Resilient
GPT
Long term bets
Covid Boom Bust
2024 Q1 short
2024 long term
2024 2dm
2024, Fall, Short Term, Long
Nuclear


In [78]:
stock_lists

{'Post Stonks': ['SMR',
  'IOT',
  'SHOP',
  'ASML',
  'ADSK',
  'RDW',
  'RKLB',
  'NOW',
  'MGA',
  'COST',
  'APTV',
  'GM',
  'DDOG',
  'IBB',
  'ENTG',
  'IVV',
  'CCI',
  'GOLF',
  'IRT'],
 'Dividend': ['JEPI',
  'XYLD',
  'RYLD',
  'SHW',
  'DGRO',
  'DLR',
  'IRM',
  'MRK',
  'VZ',
  'CTAS',
  'WSBF',
  'NEM',
  'MPLX',
  'AGNC',
  'O',
  'RWT'],
 'Biotech': ['JNJ',
  'AMGN',
  'GILD',
  'AZN',
  'PFE',
  'MRNA',
  'CYRX',
  'VRTX',
  'BMY'],
 'Energy': ['CEG', 'SMR', 'OKLO', 'COP', 'XLE'],
 'Bonds': ['IBHE', 'TIP'],
 'Banks, Finance and Homes': ['AAL',
  'GS',
  'C',
  'KBH',
  'SEDG',
  'CNXC',
  'JBGS',
  'SNV',
  'OZK',
  'WAL',
  'FITB',
  'FHN',
  'EWBC',
  'XRT',
  'USB',
  'ABCB',
  'CATY',
  'FFIN',
  'CBSH',
  'WBS',
  'FIBK',
  'LNC',
  'CRBG',
  'BAC',
  'EMB',
  'HYG',
  'SPG',
  'CACC',
  'ALLY',
  'COF',
  'RF',
  'FRCB',
  'CMBS',
  'TSLA',
  'VNO',
  'BXP',
  'NYMT',
  'DX',
  'SLG',
  'KRE'],
 'High debt load': ['VSCO', 'KMX', 'VZ', 'T', 'CMCSA', 'NFLX', 'CHTR

In [152]:
stock_lists.keys()

dict_keys(['Post Stonks', 'Dividend', 'Biotech', 'Energy', 'Bonds', 'Banks, Finance and Homes', 'High debt load', 'Massachusetts', 'Daily Movers', 'QT Resilient', 'GPT', 'Long term bets', 'Covid Boom Bust', '2024 Q1 short', '2024 long term', '2024 2dm', '2024, Fall, Short Term, Long', 'Nuclear'])

In [196]:
figs = []
slopes = []

for stock_list in stock_lists.keys():
    historical_data[stock_list] = run_stats_for_group(stock_list)
    fig, slope_df = calulate_time_slope_delta_plot(historical_data[stock_list], group_title=stock_list)
    figs.append(fig)
    slopes.append(slope_df)


#historical_data[group_name] = run_stats_for_group(group_name)
#calulate_time_slope_delta_plot(historical_data, group_title=group_name)

['SMR', 'IOT', 'SHOP', 'ASML', 'ADSK', 'RDW', 'RKLB', 'NOW', 'MGA', 'COST', 'APTV', 'GM', 'DDOG', 'IBB', 'ENTG', 'IVV', 'CCI', 'GOLF', 'IRT']
Loading from cache
Loading from cache
Loading from cache
Loading from cache
Loading from cache
Loading from cache
Loading from cache
Loading from cache
Loading from cache
Loading from cache
Loading from cache
Loading from cache
Loading from cache
Loading from cache
Loading from cache
Loading from cache
Loading from cache
Loading from cache
Loading from cache


/tmp/ipykernel_1221821/142773724.py:26: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



['JEPI', 'XYLD', 'RYLD', 'SHW', 'DGRO', 'DLR', 'IRM', 'MRK', 'VZ', 'CTAS', 'WSBF', 'NEM', 'MPLX', 'AGNC', 'O', 'RWT']
Loading from cache
Loading from cache
Loading from cache
Loading from cache
Loading from cache
Loading from cache
Loading from cache
Loading from cache
Loading from cache
Loading from cache
Loading from cache
Loading from cache
Loading from cache
Loading from cache
Loading from cache
Loading from cache
['JNJ', 'AMGN', 'GILD', 'AZN', 'PFE', 'MRNA', 'CYRX', 'VRTX', 'BMY']
Loading from cache
Loading from cache
Loading from cache
Loading from cache
Loading from cache
Loading from cache
Loading from cache
Loading from cache
Loading from cache
['CEG', 'SMR', 'OKLO', 'COP', 'XLE']
Loading from cache
Loading from cache
Loading from cache
Loading from cache
Loading from cache
['IBHE', 'TIP']
Loading from cache
Loading from cache
['AAL', 'GS', 'C', 'KBH', 'SEDG', 'CNXC', 'JBGS', 'SNV', 'OZK', 'WAL', 'FITB', 'FHN', 'EWBC', 'XRT', 'USB', 'ABCB', 'CATY', 'FFIN', 'CBSH', 'WBS', 'FIBK

In [197]:
for fig in figs:
    fig.show()

In [198]:
slopes_df = pd.concat(slopes)

In [199]:
slopes_df = slopes_df.drop_duplicates(subset=['ticker', 'time_range'])

In [200]:
slopes_df

,ticker,slope,exp_param,variance,standard_deviation,maximum_drawdown,time_range
0,SMR,0.005007,1.0,6.916598,2.629943,-9.113214,5_years
10,APTV,-0.024528,1.0,165.961378,12.882600,-44.640474,5_years
1,IOT,0.010246,1.0,28.957911,5.381255,-18.646996,5_years
18,IRT,0.002640,1.0,1.923116,1.386765,-4.805383,5_years
17,GOLF,0.019586,1.0,105.826595,10.287205,-35.646976,5_years
...,...,...,...,...,...,...,...
16,UEC,0.020333,1.0,1.042809,1.021180,-3.558287,6_months
21,UEC,-0.007201,1.0,0.032656,0.180708,-0.612062,3_months
22,CCJ,-0.036548,1.0,0.841259,0.917202,-3.106577,3_months
26,UEC,0.055000,1.0,0.002017,0.044907,-0.110000,1_week


In [205]:
def rank_slopes(slopes_df):
    # Define weights for each time period
    weights = {
        '5_years': 1.0,
        '3_years': 0.4,
        '1_year': 0.6,
        '6_months': 0.4,
        '3_months': 0.2,
        '1_week': 0.4,
    }
    
    # Ensure all time ranges in weights are present in slopes_df
    for period in weights.keys():
        if period not in slopes_df['time_range'].unique():
            raise ValueError(f"Time range '{period}' not found in slopes_df")
    
    # Calculate weighted score for each row
    slopes_df['weighted_score'] = slopes_df.apply(
        lambda row: row['slope'] * weights[row['time_range']], axis=1
    )
    
    # Aggregate weighted scores by ticker
    aggregated_df = slopes_df.groupby('ticker').agg({
        'weighted_score': 'sum',
        'variance': 'mean',
        'standard_deviation': 'mean',
        'maximum_drawdown': 'mean'
    }).reset_index()
    
    # Rank the tickers based on the aggregated weighted score
    aggregated_df['rank'] = aggregated_df['weighted_score'].rank(ascending=False)
    
    # Sort the DataFrame by rank
    ranked_df = aggregated_df.sort_values(by='rank')
    
    return ranked_df

In [255]:
def calculate_weighted_slope(df):
    """
    Calculate the weighted slope for each row in the DataFrame.
    
    Parameters:
    df (pd.DataFrame): The DataFrame containing the slope columns.
    columns (list): The list of columns to consider for the weighted slope.
    weights (dict): The dictionary of weights for each column.
    
    Returns:
    pd.Series: The weighted slope for each row.
    """

    columns = ['5_years', '3_years', '1_year', '6_months', '3_months']
    weights = {
    '5_years': 0.2,  # 50% / 4
    '3_years': 0.1,  # 50% / 4
    '1_year': 0.1,   # 50% / 4
    '6_months': 0.1, # 50% / 4
    '3_months': 0.2,    # 50%
    '1_week': 0.3,   # 50%
}

    # Ensure all columns in weights are present in df
    for col in columns:
        if col not in df.columns:
            raise ValueError(f"Column '{col}' not found in DataFrame")
    
    # Calculate the weighted slope
    weighted_slope = sum(df[col] * weights[col] for col in columns)
    
    return weighted_slope

def rank_by_momentum_shift(slopes_df):
    # Ensure the required time ranges are present in slopes_df
    required_periods = ['5_years', '3_years', '1_year', '6_months', '3_months', '1_week']
    for period in required_periods:
        if period not in slopes_df['time_range'].unique():
            raise ValueError(f"Time range '{period}' not found in slopes_df")
    
    # Pivot the DataFrame to have time ranges as columns
    pivot_df = slopes_df.pivot(index='ticker', columns='time_range', values='slope').reset_index()

    pivot_df['second_order_accel'] = calculate_weighted_slope(pivot_df)

    # Merge the pivot_df with the original slopes_df to retain other columns
    merged_df = pd.merge(slopes_df.drop(columns=['slope', 'time_range']), pivot_df, on='ticker').drop_duplicates('ticker')
    
    # Rank the tickers based on the momentum shift
    merged_df['rank'] = merged_df['second_order_accel'].rank(ascending=False)
    
    # Sort the DataFrame by rank
    ranked_df = merged_df.sort_values(by='rank')
    
    return ranked_df

In [256]:
ranked_slopes_df = rank_slopes(slopes_df)

In [257]:
ranked_slopes_df

,ticker,weighted_score,variance,standard_deviation,maximum_drawdown,rank
54,BKNG,38.073716,300342.727077,453.579802,-1562.347134,1.0
234,NFLX,6.598013,8977.173826,76.971732,-264.947383,2.0
136,EXOD,4.043809,11.433858,1.522763,-3.872547,3.0
80,CEG,3.742068,1487.531082,33.596035,-115.196269,4.0
67,CACC,3.691857,411.318016,17.196107,-58.202821,5.0
...,...,...,...,...,...,...
197,KLAC,-4.039685,8706.720997,64.341671,-221.155078,367.0
160,GOEV,-4.944295,774523.731702,455.581689,-1578.199703,368.0
32,ASML,-7.258939,6992.003994,71.600499,-245.554003,369.0
327,TPL,-7.582686,31196.663931,149.114962,-513.211150,370.0


In [258]:
ranked_momentum_shift_df = rank_by_momentum_shift(slopes_df)


In [259]:
ranked_momentum_shift_df.sort_values(['1_week', 'second_order_accel'], ascending=False)

,ticker,exp_param,variance,standard_deviation,maximum_drawdown,weighted_score,5_years,3_years,1_year,6_months,3_months,1_week,second_order_accel,rank
1866,BKNG,1.0,5.734806e+05,757.285010,-2624.125718,1.441827,1.441827,2.708695,4.645489,10.211034,-2.296481,72.840,1.585591,1.0
558,NFLX,1.0,2.709137e+03,52.049372,-180.360227,0.099099,0.099099,0.566272,0.952704,1.849513,1.204889,11.800,0.597646,3.0
1770,EXOD,1.0,7.099895e-04,0.026646,-0.092332,-0.000051,-0.000051,-0.000140,-0.001235,-0.004926,-0.016864,10.125,-0.004013,212.0
336,CACC,1.0,1.101087e+03,33.182634,-114.983661,0.063178,0.063178,-0.031859,-0.300713,0.167040,0.515172,9.130,0.099117,26.0
264,CEG,1.0,2.886064e+03,53.722103,-186.156532,0.102284,0.102284,0.193706,0.332586,0.530993,0.781764,7.485,0.282538,9.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1830,KLAC,1.0,2.777667e+04,166.663355,-577.517830,0.317317,0.317317,0.476207,-0.004763,-0.756043,0.878951,-11.045,0.210794,13.0
1794,RH,1.0,1.446024e+03,38.026618,-131.768916,-0.072401,-0.072401,0.009884,0.406005,1.055783,1.411245,-12.035,0.414936,7.0
84,ASML,1.0,1.788975e+04,133.752558,-463.476133,0.254657,0.254657,0.342556,-0.888624,-1.096385,1.065553,-17.230,0.099797,25.0
2142,TPL,1.0,4.505289e+04,212.256669,-735.506681,0.404125,0.404125,0.471423,2.722115,3.924125,-1.271496,-27.810,0.538292,5.0


In [260]:
import plotly.graph_objects as go

def plot_stock_data(stock_ticker_list, historical_df):
    fig = go.Figure()

    for ticker in stock_ticker_list:
        if ticker in historical_df:
            df = historical_df[ticker].drop(columns = ['begins_at']).reset_index()
            df = df.sort_values('begins_at')
            fig.add_trace(go.Scatter(x=df['begins_at'], y=df['close_price'].apply(pd.to_numeric), mode='lines', name=ticker))

    fig.update_layout(
        title="Stock Prices Over Time",
        xaxis_title="Date",
        yaxis_title="Price",
        legend_title="Stock Tickers"
    )

    fig.show()

# Flatten out the group name and just keep the stock_ticker:dataframe
flattened_historical_data = {ticker: df for group in historical_data.values() for ticker, df in group.items()}

In [264]:
# Assuming historical_df is a DataFrame with stock tickers as columns and dates as index
plot_stock_data(ranked_momentum_shift_df.sort_values(['second_order_accel'], ascending=False)['ticker'].tolist()[:5], flattened_historical_data)

In [263]:
# Assuming historical_df is a DataFrame with stock tickers as columns and dates as index
plot_stock_data(ranked_momentum_shift_df.sort_values(['second_order_accel'], ascending=False)['ticker'].tolist()[-5:], flattened_historical_data)